## Бортников Павел Павлович 20215 - кузнечик
https://habr.com/ru/post/459004/

In [1437]:
import binascii
test_small = "1122334455667700ffeeddccbbaa9988"
test_medium = ""
test_big = ""
with open('./files/largetext1.txt', 'r') as f:
    test_medium = f.read()
with open('./files/largetext2.txt', 'r') as f:
    test_big = f.read()

Сразу зададим коэффициенты ряда Галуа, таблицы степеней двойки для поля GF(2^8) и таблицу для нелинейного преобразования

In [1438]:
# Ряд Галуа
galua_coef = [148, 32, 133, 16, 194, 192, 1, 251, 1, 192, 194, 16, 133, 32, 148, 1]
galua_coef_reverse = galua_coef[::-1]
# Таблица степеней двойки (GF(2^8))
galua_fields = [1, 2, 4, 8, 16, 32, 64, 128, 195, 69, 138, 215, 109, 218, 119, 238, 31, 62, 124, 248, 51, 102, 204, 91,
                182, 175, 157, 249, 49, 98, 196, 75, 150, 239, 29, 58, 116, 232, 19, 38, 76, 152, 243, 37, 74, 148, 235,
                21, 42, 84, 168, 147, 229, 9, 18, 36, 72, 144, 227, 5, 10, 20, 40, 80, 160, 131, 197, 73, 146, 231, 13,
                26, 52, 104, 208, 99, 198, 79, 158, 255, 61, 122, 244, 43, 86, 172, 155, 245, 41, 82, 164, 139, 213,
                105, 210, 103, 206, 95, 190, 191, 189, 185, 177, 161, 129, 193, 65, 130, 199, 77, 154, 247, 45, 90, 180,
                171, 149, 233, 17, 34, 68, 136, 211, 101, 202, 87, 174, 159, 253, 57, 114, 228, 11, 22, 44, 88, 176,
                163, 133, 201, 81, 162, 135, 205, 89, 178, 167, 141, 217, 113, 226, 7, 14, 28, 56, 112, 224, 3, 6, 12,
                24, 48, 96, 192, 67, 134, 207, 93, 186, 183, 173, 153, 241, 33, 66, 132, 203, 85, 170, 151, 237, 25, 50,
                100, 200, 83, 166, 143, 221, 121, 242, 39, 78, 156, 251, 53, 106, 212, 107, 214, 111, 222, 127, 254, 63,
                126, 252, 59, 118, 236, 27, 54, 108, 216, 115, 230, 15, 30, 60, 120, 240, 35, 70, 140, 219, 117, 234,
                23, 46, 92, 184, 179, 165, 137, 209, 97, 194, 71, 142, 223, 125, 250, 55, 110, 220, 123, 246, 47, 94,
                188, 187, 181, 169, 145, 225, 1]
# Таблица нелинейного преобразования для прямого хода
nonlinear_coef = [252, 238, 221, 17, 207, 110, 49, 22, 251, 196, 250, 218, 35, 197, 4, 77, 233, 119, 240, 219, 147, 46,
                  153, 186, 23, 54, 241, 187, 20, 205, 95, 193, 249, 24, 101, 90, 226, 92, 239, 33, 129, 28, 60, 66,
                  139, 1, 142, 79, 5, 132, 2, 174, 227, 106, 143, 160, 6, 11, 237, 152, 127, 212, 211, 31, 235, 52, 44,
                  81, 234, 200, 72, 171, 242, 42, 104, 162, 253, 58, 206, 204, 181, 112, 14, 86, 8, 12, 118, 18, 191,
                  114, 19, 71, 156, 183, 93, 135, 21, 161, 150, 41, 16, 123, 154, 199, 243, 145, 120, 111, 157, 158,
                  178, 177, 50, 117, 25, 61, 255, 53, 138, 126, 109, 84, 198, 128, 195, 189, 13, 87, 223, 245, 36, 169,
                  62, 168, 67, 201, 215, 121, 214, 246, 124, 34, 185, 3, 224, 15, 236, 222, 122, 148, 176, 188, 220,
                  232, 40, 80, 78, 51, 10, 74, 167, 151, 96, 115, 30, 0, 98, 68, 26, 184, 56, 130, 100, 159, 38, 65,
                  173, 69, 70, 146, 39, 94, 85, 47, 140, 163, 165, 125, 105, 213, 149, 59, 7, 88, 179, 64, 134, 172, 29,
                  247, 48, 55, 107, 228, 136, 217, 231, 137, 225, 27, 131, 73, 76, 63, 248, 254, 141, 83, 170, 144, 202,
                  216, 133, 97, 32, 113, 103, 164, 45, 43, 9, 91, 203, 155, 37, 208, 190, 229, 108, 82, 89, 166, 116,
                  210, 230, 244, 180, 192, 209, 102, 175, 194, 57, 75, 99, 182]
# Таблица нелинейного преобразования для обратного хода
nonlinear_coef_reverse = [165, 45, 50, 143, 14, 48, 56, 192, 84, 230, 158, 57, 85, 126, 82, 145, 100, 3, 87, 90, 28, 96,
                          7, 24, 33, 114, 168, 209, 41, 198, 164, 63, 224, 39, 141, 12, 130, 234, 174, 180, 154, 99, 73,
                          229, 66, 228, 21, 183, 200, 6, 112, 157, 65, 117, 25, 201, 170, 252, 77, 191, 42, 115, 132,
                          213, 195, 175, 43, 134, 167, 177, 178, 91, 70, 211, 159, 253, 212, 15, 156, 47, 155, 67, 239,
                          217, 121, 182, 83, 127, 193, 240, 35, 231, 37, 94, 181, 30, 162, 223, 166, 254, 172, 34, 249,
                          226, 74, 188, 53, 202, 238, 120, 5, 107, 81, 225, 89, 163, 242, 113, 86, 17, 106, 137, 148,
                          101, 140, 187, 119, 60, 123, 40, 171, 210, 49, 222, 196, 95, 204, 207, 118, 44, 184, 216, 46,
                          54, 219, 105, 179, 20, 149, 190, 98, 161, 59, 22, 102, 233, 92, 108, 109, 173, 55, 97, 75,
                          185, 227, 186, 241, 160, 133, 131, 218, 71, 197, 176, 51, 250, 150, 111, 110, 194, 246, 80,
                          255, 93, 169, 142, 23, 27, 151, 125, 236, 88, 247, 31, 251, 124, 9, 13, 122, 103, 69, 135,
                          220, 232, 79, 29, 78, 4, 235, 248, 243, 62, 61, 189, 138, 136, 221, 205, 11, 19, 152, 2, 147,
                          128, 144, 208, 36, 52, 203, 237, 244, 206, 153, 16, 68, 64, 146, 58, 1, 38, 18, 26, 72, 104,
                          245, 129, 139, 199, 214, 32, 10, 8, 0, 76, 215, 116]

XOR функция для двух строк

In [1439]:
def xor_str(s1, s2):
    return bytes(a ^ b for a, b in zip(s1, s2))

Функция линейного преобразования блока
Каждый байт из блока умножается в поле Галуа на один из коэффициентов ряда в зависимости от порядкового номера байта. Байты складываются между собой по модулю 2, и все 16 байт блока сдвигаются в сторону младшего разряда, а полученное число записывается на место считанного байта.

In [1440]:

def galua_f(a, b):
    c = 0
    for i in range(0, 8):
        if (b % 2):
            c ^= a
        hi_bit = a & 0x80
        a = (2 * a) % 256
        if (hi_bit):
            a ^= 0xc3
        b >>= 1
    return c


def r(block):
    block = block[::-1]
    res = [0] * 16
    for i in range(15, -1, -1):
        if i > 0:
            res[i - 1] = block[i]
        res[15] ^= galua_f(block[i], galua_coef_reverse[i])
    return bytes(res[::-1])

def reverse_r(block):
    block = block[::-1]
    res = [0] * 16
    a0 = block[15]
    for i in range(16):
        if i > 0:
            res[i] = block[i - 1]
        a0 ^= galua_f(res[i], galua_coef_reverse[i])
    res[0] = a0
    return bytes(res[::-1])

def linear_transformation(block, reverse=False):
    res = block
    for i in range(16):
        # print("Reverse = ", reverse)
        if reverse:
            # print("Im in reverse")
            res = reverse_r(res)
        else:
            res = r(res)
    return bytes(res)

Функция нелинейного преобразования блока (представляет собой простую замену одного байта на другой в соответствии с таблицей)

In [1441]:
def nonlinear_transformation(num, move='straight'):
    num = binascii.hexlify(num)
    for i in range(16):
        # Выбор таблицы для хода
        if move == 'reverse':
            nonlinear_table = nonlinear_coef_reverse
        else:
            nonlinear_table = nonlinear_coef

        # Выборка пары цифр, которые будут заменены в соответствии с таблицей `nonlinear_coef` или
        # `nonlinear_coef_reverse`
        num_for_replace = num[i * 2: i * 2 + 2]
        convert_num = int(num_for_replace, 16)
        num_for_replace = hex(nonlinear_table[convert_num])[2:]
        if len(num_for_replace) == 1:
            num_for_replace = '0' + num_for_replace

        # Возврат числа
        num = b"".join([num[: i * 2], binascii.hexlify(bytearray.fromhex(num_for_replace)), num[i * 2 + 2:]])
    return binascii.unhexlify(num)

Отметим именование функций

In [1442]:
X = xor_str
S = nonlinear_transformation
L = linear_transformation

Функция получения пачки ключей для энкрипта/декрипта

In [1443]:
def get_keys(key):
    while len(key) < 64:
        key += key
    key = key[:64]

    # print("key:", key, binascii.unhexlify(key))
    key = binascii.unhexlify(key)
    C = []  # константы
    K = [0 for i in range(10)]
    K[0] = key[:int(len(key) / 2)]
    K[1] = key[int(len(key) / 2):]
    for i in range(32):
        num = b'\x00' * 31 + bytes([i + 1])
        C.append(L(num))


    # формирование ячеек Фейстеля
    def F (in_key_1, in_key_2, const):
        out_key_2 = in_key_1
        out_key_1 = X(L(S(X(in_key_1, const))), in_key_2)
        return [out_key_1, out_key_2]
    iter12 = [0, 0]
    iter34 = [0, 0]
    iter12[0] = K[0]
    iter12[1] = K[1]
    for i in range(4):
        for j in range(0, 8, 2):
            iter34 = F(iter12[0], iter12[1], C[j + 8*i])
            iter12 = F(iter34[0], iter34[1], C[j+1 + 8*i])

        K[2 * i + 2] = iter12[0]
        K[2 * i + 3] = iter12[1]
    return K

Ну и завершающие функции самого энкрипта и декрипта по паролю

In [1444]:
def encrypt(text, passcode):
    # text = binascii.hexlify(bytearray.fromhex(text.encode('utf-8').hex()))
    text = binascii.unhexlify(text)
    # print("Converted: ", text)
    keys = get_keys(passcode)

    count = 32 - len(text) % 32
    if count != 0 and count != 32:
        for i in range(count):
            text += b'\x00'
    # print(text)
    textArray = []
    for i in range(int(len(text) / 32)):
        textArray.append(text[i * 32: i * 32 + 32])

    textEncrypt = []
    for j in textArray:
        # шифрование текста
        textEncrypted = j
        for i in range(9):
            textEncrypted = L(S(X(textEncrypted, keys[i])))
        textEncrypted = X(textEncrypted, keys[9])
        textEncrypt.append(textEncrypted)
        # print(textEncrypt)
    res = bytes()
    for x in textEncrypt:
        res += binascii.hexlify(x)
    return res

# encrypt("1122334455667700ffeeddccbbaa9988", "8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef")

In [1445]:
def decrypt(text, passcode):
    keys = get_keys(passcode)

    textArray = []
    for i in range(int(len(text) / 32)):
        textArray.append(text[i * 32: i * 32 + 32])
    # print(textArray)
    textDecrypt = []
    for j in textArray:
        # расшифровка текста
        textDecrypted = binascii.unhexlify(j)
        for i in range(9, 0, -1):
            textDecrypted = S(L(X(textDecrypted, keys[i]), True), 'reverse')
        textDecrypted = X(textDecrypted, keys[0])
        textDecrypt.append(textDecrypted)
    res = bytes()
    # print(textDecrypt)
    for x in textDecrypt:
        res += binascii.hexlify(x)
    return res
    # return bytes.fromhex(''.join(textDecrypt)).decode('utf-8')

# Тесты

In [1446]:
passcode = "8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef"
encrypted = encrypt(test_small, passcode)
decrypted = decrypt(encrypted, passcode)
print(test_small + ' --> ', encrypted, '-->', decrypted)

1122334455667700ffeeddccbbaa9988 -->  b'7f679d90bebc24305a468d42b9d4edcd' --> b'1122334455667700ffeeddccbbaa9988'


In [1447]:
test_medium = test_medium.encode('utf-8').hex()
encrypted = encrypt(test_medium, passcode)
print("Encrypted, starting decrypt")
decrypted = decrypt(encrypted, passcode)

4c6f72656d20697073756d20646f6c
Encrypted, starting decrypt


In [1448]:
print(binascii.unhexlify(decrypted).decode('utf-8')) #220 кб Считает за 30 секунд

Lorem ipsum doloectetur adipiscieiusmod tempor iore et dolore mat scelerisque inectetur a erat. onec enim diam vquet lectus proiimentum id venenvida cum sociis s et magnis dis.tristique senectInterdum consectaucibus nisl tinam. Aliquam purus. Fringilla ut augue interdum vles ut etiam sitrci sagittis eu cilisis mauris. se sed nisi lacut erat pellentesommodo elit at iam nulla porttite aliquam vestibissim diam quis elerisque fermen. Turpis egestasiquet.

In nisl  eu ultrices vitlor sit amet coning elit duis. Tconvallis aeneanaculis at erat piscing commodo. sse potenti nullementum nibh telc non blandit maies integer quis. Varius vel pha nunc eget lorem massa vitae tor Adipiscing elits sit amet luctuus. Nascetur rids vitae ultricieUrna nunc id curo in vitae turpi turpis cursus iplatea. Feugiat bulum mattis ullit amet nisl sus bibendum est. Es a pellentesquetor eget dolor. gestas pretium aAmet mattis vulp aliquet. Tristiuis ipsum suspenravida dictum fudo viverra maeceus vel. Eu mi bistas con